In [1]:
import pandas as pd
import numpy as np
import requests
import hopsworks
import joblib

In [ ]:
%load_ext autoreload
%autoreload 2

# Importing Air Qualtiy Dataset for Gothenburg year 2023
##### From https://goteborg.se/wps/portal/start/kommun-och-politik/sa-arbetar-goteborgs-stad-med/digitalisering/oppna-data/sok-oppna-data/oppna-data---datamangd#esc_entry=690&esc_context=6

# EDA

In [67]:
from air_pred.utils import data_preprocessing
import great_expectations as ge
def value_not_null(expectation_suite, column_name):
    expectation_suite.add_expectation(ge.core.ExpectationConfiguration(
                                        expectation_type="expect_column_values_to_not_be_null",
                                        kwargs={
                                            "column": column_name,
                                        }
                                        ))

In [68]:
fg = fs.get_or_create_feature_group(name="air_quality_data",
                                        version=FEAURE_GROUP_VERSION,
                                        description="Uncleaned raw data for air quality in Gothenburg",
                                        online_enabled=False,
                                        primary_key=["date_time"],
                                        event_time='date_time')
df = pd.read_csv("air_quality_2023.csv", skipinitialspace = True)
processed_df = data_preprocessing.create_date_time_feature(df)
#fg.insert(processed_df, wait=True, write_options={"wait_for_job":True}

In [79]:
clean_data_fg = fs.get_or_create_feature_group(name="cleaned_air_quality_data",
                                            version=FEAURE_GROUP_VERSION,
                                            description="Cleaned raw data for air quality in Gothenburg",
                                            online_enabled=True,
                                            primary_key=["date_time_str"],
                                            event_time='date_time')
expectation_suite = ge.core.ExpectationSuite(expectation_suite_name="cleaned_air_quality_fg")

cleneddf = data_preprocessing.clean_data(processed_df)

for feature in cleneddf.columns:
    value_not_null(expectation_suite, feature)
    
if clean_data_fg.get_expectation_suite is None:
    clean_data_fg.save_expectation_suite(expectation_suite, run_validation=True, validation_ingestion_policy="STRICT")
# Must do backfill also

clean_data_fg.insert(cleneddf, wait=True, write_options={"wait_for_job":True})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/140436/fs/140355/fg/159091


Uploading Dataframe: 0.00% |          | Rows 0/7089 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: cleaned_air_quality_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/140436/jobs/named/cleaned_air_quality_data_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x167118d90>, None)

In [80]:
es = clean_data_fg.get_expectation_suite()

In [81]:
es